In [3]:
#testing git skills 6:52pm
import requests
from bs4 import BeautifulSoup

#read initialisation file (for login/passwords)
import configparser
config = configparser.ConfigParser()
config.read('xrefusernames.ini')

#open googlesheet for prices
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope=['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

#My understanding is the GoogleSheets Client email isn't sensitive information - the JSON 
#file associated with it IS. So don't upload the JSON file.
credentials = ServiceAccountCredentials.from_json_keyfile_name("Price Lists-4a1d16d29c20.json", scope)

#google credentials
gc = gspread.authorize(credentials)

#wks =  the 'comparison' sheet (sheet1) on the PRICE LIST spreadsheet
wks = gc.open('PRICE LIST').sheet1

###Update Cell B2 with the value and return price
wks.update_acell('B2','WCF31NM')
print(wks.acell('C2').value)
                                                               



def BaldwinXref(part, xreflist):
    #!!!capital letters don't work properly :/

    #remove leading zeros as Baldwin site can't handle them
    while part[0]=='0':
        part=part[1:]
    #print(part)
    
    r=requests.get("http://catalog.baldwinfilter.com/BaldwinDisplay.asp?URL=BaldwinOEM.asp&partnumber="+part+"=&optiontype=OEM")
    data=r.text
   # print(data)
    soup = BeautifulSoup(data, "lxml")

    links = soup.find_all("a")


    if(len(links)==1):
        xreflist.append(["No Xref in", "Baldwin"]) 
    else:
        for j in range(len(links)-2):
            xreflist.append([links[j].text, "BALDWIN"])
    
    #print results
#    for item in xreflist:
#        print(*item)
    return xreflist
       
                
                
def DonaldsonXref(part,xreflist):
    
    p = {'txt_partnumber': part,
     'btnPartSearch': 'find'}
  
    #xreflist = [[]] #list of lists to collect x-references

    r = requests.post("http://donaldsontoolbox.com.au/search/crossreference/getAjaxParts/",p)
    #print(r.content)
    
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    info = soup.find_all("td")

    #Check to see if part is found
    notFound = data.partition('part not found')[1]
    if(notFound=='part not found'):
        xreflist.append(["No xref in","Donaldson"])
    else:
        counter = 10
        while(counter<len(info)):
            xreflist.append([info[counter].text, "DONALDSON"])
            counter=counter+6
    
    #debugging -print list
    #for item in xreflist:
    #    print(*item)
        
    return xreflist
        
        
def MannXref(part, xreflist):
    r = requests.get("https://catalog.mann-filter.com/EU/eng/oenumbers")
   # print(r.content)
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    
#retreive view state
    myState = soup('input', attrs={'name':"javax.faces.ViewState"})[0]['value']

#specify j_idt - this changes each time? either j_idt180 or j_idt132
    myJIDT = 'j_idt132'
#LOOP############################################################################    
    while True:
        #https://catalog.mann-filter.com/EU/eng/oenumbers
        payload = {
    'autocompleteOEsearch:autocompleteOEsearch':'autocompleteOEsearch:autocompleteOEsearch',
    'autocompleteOEsearch:autocompleteOEsearch:searchQueryValue':part,
    'autocompleteOEsearch:autocompleteOEsearch:searchQueryInput':part,
    'javax.faces.ViewState':myState,
    'javax.faces.source':'autocompleteOEsearch:autocompleteOEsearch:'+myJIDT,
    'javax.faces.partial.execute':'autocompleteOEsearch:autocompleteOEsearch:'+myJIDT+' @component',
    'javax.faces.partial.render':'@component',
    'searchQuery':part,
    'org.richfaces.ajax.component':'autocompleteOEsearch:autocompleteOEsearch:'+myJIDT,
    'autocompleteOEsearch:autocompleteOEsearch:'+myJIDT:'autocompleteOEsearch:autocompleteOEsearch:'+myJIDT,
    'rfExt':'null',
    'AJAX:EVENTS_COUNT':'1',
    'javax.faces.partial.ajax':'true',
    }
    
        r = requests.post("https://catalog.mann-filter.com/EU/eng/oenumbers",payload) #from clicking x-ref in navigation bar
        #print(r.content)
        data = r.text
        soup = BeautifulSoup(data, "lxml")
    
#DEBUGGING
  #  print(soup.prettify())
#/DEBUGGING

        parts_list = soup.find_all('div', attrs={'data-label':"MANN-FILTER"})
#debugging    
    #print("parts_list length is: "+str(len(parts_list)))
#/debugging
        if(len(parts_list)==0): #item not found
            #xreflist.append(["No xref in","MANN"])
            myJIDT = 'j_idt180'
        else:
            parts_list.pop(0) #was this because first result was always not required??
            break
#######################################################################end of LOOP    
    if(len(parts_list)==0): #item not found
        xreflist.append(["No xref in","MANN"])

    

#transfer link-text to xreflist
    for x in parts_list: 
        if(repr(x.text)=="\'\\nNo equivalent\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\'"):
            xreflist.append(["Part number known but no xref in","MANN"])

        else:
            temp = repr(x.text)
            temp = temp.replace("\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\n\'","")
         #   print("*debug* current TEMP value is: "+temp)
            temp = temp.replace("\'\\n\\n","")
            temp = temp.replace(' ','')
            xreflist.append([temp, "MANN"])
#DEBUGGING
#print cross references
#    for item in xreflist:
#    print(*item)
  
    return xreflist
    
            
def WixXref(part, xreflist):
#!!!capital letters don't work properly :/
   # part="wp111023"

    r=requests.get("http://www.wixfilters.com/Lookup/InterchangeMultiSearch.aspx?q="+part+"&o=me")
    data=r.text
    soup = BeautifulSoup(data, 'lxml')
#debugging
    #print(soup.prettify())
    
    #*****This can be improved?? Dig deeper into html***********************
    links = soup.find_all("a")
    
    #was 66
    #for j in range(28,len(links)-13):
    for j in range(64,len(links)-13):
        if(links[j].text == "There is no information available on the part number you submitted."):
            xreflist.append(["No xref in", "WIX"])
            break
        else:
            xreflist.append([links[j].text, "WIX"])
    
    #dDEBUGGING - print list   
    #for item in xreflist:
    #    print(*item)
    
    return xreflist
        
def WesfilXref(part, xreflist):
    session_requests = requests.session()
    
   # config['wesfil']['username']
    
    payload = {
        'user[email_address]': config['wesfil']['user'],
        'user[password]': config['wesfil']['password'],
        'login':'',
        'redirect_uri':'/login',
        'on_failure':'render:/login',
        'on_success':'render:/account',
              }
    
    login_url = "http://www.wesfil.com.au/login"
    result = session_requests.get(login_url)
    
    result = session_requests.post(
        login_url, 
        data = payload, 
        headers = dict(referer=login_url)
    )
    
    url = 'http://www.wesfil.com.au/search?q='+part+'&submit='
    result = session_requests.get(url, headers = dict(referer = url))
    data=result.text
    soup = BeautifulSoup(data, "lxml")
    
    #print(soup.prettify())
        
    parts_list = soup.find_all('div', attrs={'class':"grid-code"})
    
    #xreflist =[[]] #define empty list for collecting clened xref
    
    if(len(parts_list)==0):
        #print("No Wesfil x-ref")
        xreflist.append(["No xref in","Wesfil"])
    else:
        for x in parts_list:
            xreflist.append([x.text,"Wesfil"])
            #print(x.text)
   
    #DEBUGGING - print parts list
    #for item in xreflist:
    #    print(*item) 
        
    return xreflist
            
def SakuraXref(part, xreflist):
    session_requests = requests.session()
    
    #part = "wz79"
    
    #payload is AFTER you've logged in. Below info is from the POST
    payload = {
        '__EVENTTARGET':'',
        '__EVENTARGUMENT':'',
        '__VIEWSTATE':'/wEPDwULLTExMjI0NjQzODAPZBYCZg9kFgICAw9kFggCBw8PFgIeBFRleHQFBUxvZ2luZGQCDQ8PFgIfAAUsUGxlYXNlIGVudGVyIHlvdXIgYWNjb3VudCBjb2RlIGFuZCBwYXNzd29yZC5kZAIPD2QWAgIPDw8WAh4LTmF2aWdhdGVVcmwFtwFtYWlsdG86cGFzc3dvcmRAZnNhZmlsdGVycy5jb20uYXU/c3ViamVjdD1XZWJzaXRlIFBhc3N3b3JkIFJlcXVlc3QmYm9keT0qKiogVG8gcmVjb3ZlciB5b3VyIHBhc3N3b3JkIHBsZWFzZSBpbmNsdWRlIHlvdXIgbmFtZSwgZW1haWwgYWRkcmVzcywgY29tcGFueSBuYW1lIGFuZCBhY2NvdW50IGNvZGUgYmVsb3cuICoqKiBkZAIVDw8WAh8ABSMyMDA5IE1BUktJTlNPTiBURUNITk9MT0dJRVMgUFRZIExURGRkZNskRCorcjHCLq+dXMZuxTHo47N+',
        '__VIEWSTATEGENERATOR':'C2EE9ABB',
        '__EVENTVALIDATION':'/wEWBAKxl52QCALKhObCCwKsltaTCgK+j533D+a0WFZaw/jaULDhnAsewOTIjAyE',
        'ctl00$ContentPlaceHolder1$TextBox_Code': config['sakura']['user'],
        'ctl00$ContentPlaceHolder1$TextBox_Password': config['sakura']['password'],
        'ctl00$ContentPlaceHolder1$Button_Login':'  Login',
              }
    
    login_url = "http://www.fsafilters.com.au:81/login.aspx"
    result = session_requests.get(login_url)
    
    result = session_requests.post(
        login_url, 
        data = payload, 
        headers = dict(referer=login_url)
    )
    
    url = 'http://www.fsafilters.com.au:81/productsearch.aspx?page=search&srch='+part
    result = session_requests.get(url, headers = dict(referer = url))
    data=result.text
    soup = BeautifulSoup(data, "lxml")
    
    #print(soup.prettify())
    
    #define an empty list 
    parts_list=["","","","","","","","","",""]
    
    #find resulting part numbers. Each one is in ctl00, ctl01....., ctl109
    for i in range(0,10):
        parts_list[i] = soup.find('span', attrs={'id':"ctl00_ContentPlaceHolder1_DataList1_ctl0"+str(i)+"_ProductSnapShot1_ProductCode"})
    
   # xreflist=[[]] #define a list of lists

    if(soup.find('span', attrs={'id':"ctl00_PageDescription_label"}).text=="Your search for "+part+" returned no results."):
    #if(parts_list[0]==""):
        #print("No Sakura x-ref")
        xreflist.append(["No xref in","Sakura"])
        print("FAIL")
    else:
        x=0 #for indexing purposes
        for item in parts_list:
            if(parts_list[x]==None):
                break
            xreflist.append([parts_list[x].text,"Sakura"])
            #print(parts_list[x].text)
            x=x+1
    
    
   #DEBUGGING - print parts list
    #for item in xreflist:
    #    print(*item)
        
    return xreflist
                
        
        
def FleetguardXref(part, xreflist):
    
    r=requests.get("https://catalog.cumminsfiltration.com/catalog/catalogLeftNavSearch.do?methodName=crossRefSearchAjax&searchString="+part+"&productTypeList=&filterType=null")
    
    result = r.text
    #print(result)
    
    #print("\nFleetguard: ")
    invalidPart = result.partition("Invalid Part No")[1]
    
   # xreflist=[[]] #list of list for capturing xref    
    
    if(invalidPart=="Invalid Part No"):
        xreflist.append(["No xref in","Fleetguard"])
        #print("No x-ref found")
    else:
        numResults = result.partition("{\"iTotalRecords\":")[2][0]
    
    #define an empty list 
        parts_list=["","","","","","","","","",""]
        parts_pointer = 2
        

    
        for i in range(0,int(numResults)):
            parts_list[i] = result.split(part)[parts_pointer][3:] #split on part, skip every second one, cut off first 3 chars
            parts_list[i] = parts_list[i].split("\"")[0]
            xreflist.append([parts_list[i],"Fleetguard"])
            #print(parts_list[i])
            parts_pointer = parts_pointer+2
            
   #DEBUGGING - prints parts list
    #for item in xreflist:
    #    print(*item)
        
    return xreflist
            
def cleaner(thing):
    thing=thing.upper() #capitalise
    thing=thing.replace(' ','')
    thing=thing.replace('/','') 
    thing=thing.replace('-','')
    return thing
            
        
if __name__ == '__main__':

    XREF = [[]]  #Initiliase list which will be basis for XREF (list which collects all xref)
    
    while True:
        part = input("Enter Part: ")
        part = cleaner(part)
        supplier = input("Search which supplier? Type Selection then ENTER....\n a=ALL b=BALDWIN, d=DONALDSON, m=MANN, wix=WIX, wesfil=WESFIL, s=SAKURA, f=FLEETGUARD\n or x to exit")
        if(supplier=='x'):
            break
        
        elif (supplier=='a'or supplier=='A'):
            XREF = BaldwinXref(part, XREF)
            XREF = DonaldsonXref(part, XREF)
            XREF = MannXref(part, XREF)  
            XREF = WixXref(part, XREF)   #needs fixing not printing parts    
            XREF = WesfilXref(part, XREF)
            XREF = SakuraXref(part, XREF)  #scenario to fix: no xref
            XREF = FleetguardXref(part, XREF) 
            
        elif (supplier=='b'or supplier=='B'):
            XREF = BaldwinXref(part, XREF)
        elif(supplier=='d'or supplier=='D'):
            XREF = DonaldsonXref(part, XREF)
        elif(supplier=='m'or supplier=='m'):
            XREF = MannXref(part, XREF)  
        elif(supplier=='wix'or supplier=='WIX'):
            XREF = WixXref(part, XREF)   #needs fixing not printing parts
        elif(supplier=='wesfil'or supplier=='WESFIL'):    
            XREF = WesfilXref(part, XREF)
        elif(supplier=='s'or supplier=='S'):
            XREF = SakuraXref(part, XREF)  #scenario to fix: no xref
        elif(supplier=='f'or supplier=='F'):
            XREF = FleetguardXref(part, XREF)
    
        for item in XREF:
            print(*item)
        XREF.clear() 

$5.27


KeyboardInterrupt: 